In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
from numpy import mean
from numpy import std
from tensorflow.keras import layers
import tensorflow.keras as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

import datetime

Using TensorFlow backend.


In [6]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [7]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

In [8]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [9]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = tf.keras.utils.to_categorical(trainy)
	testy = tf.keras.utils.to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [10]:
trainX, trainy, testX, testy = load_dataset()
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [11]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq = 2000)

In [12]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, model):
	verbose, epochs, batch_size = 1, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[tensorboard_callback])
	# evaluate model
	loss, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [13]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [14]:
# run an experiment
scores = list()
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy, model)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)


In [13]:
# run the experiment
run_experiment()
# summarize results
summarize_results(scores)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Train on 7352 samples
Epoch 1/15
7352/7352 [==============================] - 14s 2ms/sample - loss: 0.1874 - accuracy: 0.9335
Epoch 2/15
7352/7352 [==============================] - 17s 2ms/sample - loss: 0.1750 - accuracy: 0.9353
Epoch 3/15
7352/7352 [==============================] - 21s 3ms/sample - loss: 0.1648 - accuracy: 0.9392
Epoch 4/15
7352/7352 [==============================] - 22s 3ms/sample - loss: 0.1552 - accuracy: 0.9415
Epoch 5/15
7352/7352 [==============================] - 21s 3ms/sample - loss: 0.1378 - accuracy: 0.9463
Epoch 6/15
7352/7352 [==============================] - 24s 3ms/sample - loss: 0.1308 - accuracy: 0.9497
Epoch 7/15
7352/7352 [==============================] - 28s 4ms/sample - loss: 0.1256 - accuracy: 0.9504
Epoch 8/15
7352/7352 [==============================] - 27s 4ms/sample - loss: 0.1297 - accuracy: 0.9490
Epoch 9/15
7352/7352 [================

In [41]:
from talos.model.normalizers import lr_normalizer
trainX, trainy, testX, testy = load_dataset()
import talos as ta


(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model = Sequential()
#model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
#model.add(Dropout(0.5))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(n_outputs, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
def tuning(x_train, y_train, x_val, y_val, params):
    # next we can build the model exactly like we would normally do it
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    # then we finish again with completely standard Keras way
    model.add(Dense(100, activation=params['activation'])
    model.add(Dense(n_outputs, activation=params['last_activation'])
    model.compile(loss=params['losses'],optimizer=params['optimizer'](lr=lr_normalizer(params['lr'],params['optimizer'])),metrics=['acc'])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=1)
    
    # finally we have to make sure that history object and model are returned
    return history, model

In [56]:
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
def tuning(x_train, y_train, x_val, y_val, params):

    # next we can build the model exactly like we would normally do it
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    
    model.add(Dropout(params['dropout']))
    
   
    # then we finish again with completely standard Keras way
    model.add(Dense(100, activation=params['activation'],
                    kernel_initializer='normal'))
    model.add(Dense(n_outputs, activation=params['last_activation'],
                    kernel_initializer='normal'))
    
    model.compile(loss=params['losses'],
                  # here we add a regulizer normalization function from Talos
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'],params['optimizer'])),
                  metrics=['acc'])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=1)
    
    # finally we have to make sure that history object and model are returned
    return history, model

In [73]:
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.losses import logcosh, categorical_crossentropy
from tensorflow.keras.activations import relu, elu, sigmoid
p = {'lr': (0.5, 5, 10),
     'batch_size': (2, 30, 10),
     'epochs': [15],
     'dropout': (0,0.5, 5),
     'optimizer': [Adam],
     'losses': [categorical_crossentropy],
     'activation':[relu, elu],
     'last_activation': [sigmoid]}

In [ ]:
t = ta.Scan(x=trainX,
            y=trainy,
            model=tuning, 
            params=p, experiment_name = 'har')


















  0%|          | 0/1000 [00:00<?, ?it/s]

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 [==============================] - 109s 21ms/sample - loss: 1.3121 - acc: 0.4030 - val_loss: 1.2630 - val_acc: 0.3985
Epoch 2/15
5146/5146 [==============================] - 114s 22ms/sample - loss: 1.0725 - acc: 0.5255 - val_loss: 1.0414 - val_acc: 0.5694
Epoch 3/15
5146/5146 [==============================] - 135s 26ms/sample - loss: 0.9047 - acc: 0.6075 - val_loss: 0.7875 - val_acc: 0.6782
Epoch 4/15
5146/5146 [==============================] - 127s 25ms/sample - loss: 0.6791 - acc: 0.7040 - val_loss: 0.7458 - val_acc: 0.6614
Epoch 5/15
5146/5146 [==============================] - 127s 25ms/sample - loss: 0.4851 - acc: 0.8022 - val_loss: 0.2723 - val_acc: 0.8939
Epoch 6/15
5146/5146 [==============================] - 116s 23ms/sample - loss: 0.2829 - acc: 0.8953 - val_loss: 0.2505 - val_acc: 0.8976
Epoch 7/15
5146/5146 [==============================] - 116s 23ms/sample - loss: 0.2074 - acc: 0.9217 - val_loss: 0.18


















  0%|          | 1/1000 [29:32<491:51:49, 1772.48s/it]

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 [==============================] - 108s 21ms/sample - loss: 1.1987 - acc: 0.4613 - val_loss: 0.8748 - val_acc: 0.5739
Epoch 2/15
5146/5146 [==============================] - 119s 23ms/sample - loss: 0.8508 - acc: 0.6125 - val_loss: 0.6658 - val_acc: 0.6759
Epoch 3/15
5146/5146 [==============================] - 113s 22ms/sample - loss: 0.6093 - acc: 0.7184 - val_loss: 0.4447 - val_acc: 0.7983
Epoch 4/15
5146/5146 [==============================] - 111s 22ms/sample - loss: 0.3654 - acc: 0.8564 - val_loss: 0.2034 - val_acc: 0.9125
Epoch 5/15
5146/5146 [==============================] - 114s 22ms/sample - loss: 0.2915 - acc: 0.8795 - val_loss: 0.2720 - val_acc: 0.9030
Epoch 6/15
5146/5146 [==============================] - 123s 24ms/sample - loss: 0.2820 - acc: 0.8921 - val_loss: 0.3618 - val_acc: 0.8622
Epoch 7/15
5146/5146 [==============================] - 118s 23ms/sample - loss: 0.2254 - acc: 0.9172 - val_loss: 0.17


















  0%|          | 2/1000 [1:00:27<498:11:48, 1797.10s/it]

...

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 [==============================] - 121s 23ms/sample - loss: 1.2751 - acc: 0.4370 - val_loss: 0.9838 - val_acc: 0.5653
Epoch 2/15
5146/5146 [==============================] - 119s 23ms/sample - loss: 0.7712 - acc: 0.6671 - val_loss: 0.5771 - val_acc: 0.7829
Epoch 3/15
5146/5146 [==============================] - 118s 23ms/sample - loss: 0.5609 - acc: 0.7820 - val_loss: 0.3793 - val_acc: 0.8645
Epoch 4/15
5146/5146 [==============================] - 120s 23ms/sample - loss: 0.3828 - acc: 0.8525 - val_loss: 0.2207 - val_acc: 0.9189
Epoch 5/15
5146/5146 [==============================] - 119s 23ms/sample - loss: 0.2585 - acc: 0.9067 - val_loss: 0.3011 - val_acc: 0.8776
Epoch 6/15
5146/5146 [==============================] - 118s 23ms/sample - loss: 0.2097 - acc: 0.9238 - val_loss: 0.1556 - val_acc: 0.9275
Epoch 7/15
5146/5146 [==============================] - 120s 23ms/sample - loss: 0.1889 - acc: 0.9240 - val_loss: 0.16


















  2%|▏         | 21/1000 [10:41:34<482:25:43, 1774.00s/it]

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 [==============================] - 121s 24ms/sample - loss: 1.2262 - acc: 0.4485 - val_loss: 0.9981 - val_acc: 0.5041
Epoch 2/15
5146/5146 [==============================] - 121s 24ms/sample - loss: 0.8662 - acc: 0.5966 - val_loss: 0.8575 - val_acc: 0.6292
Epoch 3/15
5146/5146 [==============================] - 120s 23ms/sample - loss: 0.7931 - acc: 0.6424 - val_loss: 0.7450 - val_acc: 0.6133
Epoch 4/15
5146/5146 [==============================] - 122s 24ms/sample - loss: 0.7264 - acc: 0.6522 - val_loss: 0.7270 - val_acc: 0.6351
Epoch 5/15
5146/5146 [==============================] - 120s 23ms/sample - loss: 0.7385 - acc: 0.6411 - val_loss: 0.7298 - val_acc: 0.6405
Epoch 6/15
5146/5146 [==============================] - 121s 23ms/sample - loss: 0.9562 - acc: 0.5608 - val_loss: 0.6872 - val_acc: 0.6364
Epoch 7/15
5146/5146 [==============================] - 122s 24ms/sample - loss: 0.6384 - acc: 0.6889 - val_loss: 0.64


















  2%|▏         | 22/1000 [11:12:00<486:08:37, 1789.49s/it]

Train on 5146 samples, validate on 2206 samples
Epoch 1/15
5146/5146 [==============================] - 123s 24ms/sample - loss: 1.3527 - acc: 0.3966 - val_loss: 1.3496 - val_acc: 0.4180
Epoch 2/15
5146/5146 [==============================] - 122s 24ms/sample - loss: 1.1682 - acc: 0.4854 - val_loss: 0.7601 - val_acc: 0.6491
Epoch 3/15
5146/5146 [==============================] - 122s 24ms/sample - loss: 0.7099 - acc: 0.6463 - val_loss: 0.6153 - val_acc: 0.6772
Epoch 4/15
5146/5146 [==============================] - 123s 24ms/sample - loss: 0.7498 - acc: 0.6378 - val_loss: 0.6272 - val_acc: 0.6546
Epoch 5/15
5146/5146 [==============================] - 122s 24ms/sample - loss: 0.5712 - acc: 0.7314 - val_loss: 0.6808 - val_acc: 0.7094
Epoch 6/15
5146/5146 [==============================] - 123s 24ms/sample - loss: 0.3167 - acc: 0.8828 - val_loss: 0.1591 - val_acc: 0.9325
Epoch 7/15
5146/5146 [==============================] - 124s 24ms/sample - loss: 0.2082 - acc: 0.9262 - val_loss: 0.17